### Install pandas 
Make sure to prepare the env by running the following comman

In [1]:
!pip install pandas


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


### Data Cleaning and dataframe preparation

In [11]:
import pandas as pd
#loading csv files int DFs
channels_df = pd.read_csv ('channel.csv')
subscriptions_df = pd.read_csv ('subscriptions.csv')
users_df = pd.read_csv ('users.csv')
watch_df = pd.read_csv ('watch.csv') 
# cleaning DFs 
channels_df.drop(columns=channels_df.columns[0], axis=1, inplace=True)
subscriptions_df.drop(columns=subscriptions_df.columns[0], axis=1, inplace=True)
users_df.drop(columns=users_df.columns[0], axis=1, inplace=True)
watch_df.drop(columns=watch_df.columns[0], axis=1, inplace=True)


### Preppeing TV and no TV users clusters

In [12]:
users_df = users_df.drop_duplicates(subset=['ID'])
no_tv_users_df = watch_df[watch_df['DEVICE_TYPE'] != 'tv']
tv_users_df = watch_df[watch_df['DEVICE_TYPE'] == 'tv']

### Number of tv users grouped by program kind 

In [63]:
tv_users_df.groupby('PROGRAM_KIND')['USER_ID'].nunique().sort_values(ascending=False)


PROGRAM_KIND
Informations     4687
Séries           3145
Films            3102
Documentaires    2733
Sport            2638
Indéterminé      1675
Culture          1457
Enfants          1146
Name: USER_ID, dtype: int64

### Number of non tv users grouped by program kind 

In [13]:
no_tv_users_df.groupby('PROGRAM_KIND')['USER_ID'].nunique().sort_values(ascending=False)

PROGRAM_KIND
Informations     4002
Sport            2451
Films            2232
Séries           2076
Documentaires    1815
Indéterminé       954
Culture           913
Enfants           482
Name: USER_ID, dtype: int64

### Number of non tv users grouped by asset type

In [65]:

no_tv_users_df.groupby('ASSET_TYPE')['USER_ID'].nunique().sort_values(ascending=False)

ASSET_TYPE
live                 6119
vod                  1721
replay               1061
bookmark              352
continue_watching     309
Name: USER_ID, dtype: int64

### Number of tv users grouped by asset type

In [66]:
tv_users_df.groupby('ASSET_TYPE')['USER_ID'].nunique().sort_values(ascending=False)

ASSET_TYPE
live                 5986
vod                  1660
replay               1349
bookmark              518
continue_watching     306
Name: USER_ID, dtype: int64

### Merging DFs to get subscription type by user

In [14]:
tv_merged_df = tv_users_df.merge(subscriptions_df, left_on='USER_ID', right_on='USER_ID')
no_tv_merged_df = no_tv_users_df.merge(subscriptions_df, left_on='USER_ID', right_on='USER_ID')

### Number of tv users grouped by product type

In [68]:

tv_merged_df.groupby([ 'PRODUCT_NAME'])['USER_ID'].nunique().sort_values(ascending=False)

PRODUCT_NAME
Molotov Plus        2068
Molotov Extra       1746
Molotov Giga        1184
Molotov Extended     799
Name: USER_ID, dtype: int64

### Number of non tv users grouped by product type

In [15]:
no_tv_merged_df.groupby([ 'PRODUCT_NAME'])['USER_ID'].nunique().sort_values(ascending=False)

PRODUCT_NAME
Molotov Plus        1353
Molotov Extra        993
Molotov Giga         876
Molotov Extended     580
Name: USER_ID, dtype: int64

In [16]:
from datetime import datetime
# calculating age of users 
users_df = users_df.dropna(subset=['BIRTHDAY'])
current_year = datetime.now().year
users_df['BIRTHDAY'] = pd.to_datetime(users_df['BIRTHDAY'], format='mixed')
users_df['AGE'] = current_year - users_df['BIRTHDAY'].dt.year

In [17]:
# merging tv/non-tv users with user's age
tv_demographics = pd.merge(users_df, tv_users_df, left_on='ID', right_on='USER_ID', how='inner')
no_tv_demographics = pd.merge(users_df, no_tv_users_df, left_on='ID', right_on='USER_ID', how='inner')

In [19]:
# Prepping age clusters
bins = [18, 30, 40, 50, float('inf')]
labels = ['18-30', '31-40', '41-50', '51+']

# Create age bins
tv_demographics['AGE_GROUP'] = pd.cut(tv_demographics['AGE'], bins=bins, labels=labels, right=False)
no_tv_demographics['AGE_GROUP'] = pd.cut(no_tv_demographics['AGE'], bins=bins, labels=labels, right=False)



### TV users age groups 

In [81]:
tv_demographics.groupby('AGE_GROUP').size()

C:\Users\dell\AppData\Local\Temp\ipykernel_1800\3772423047.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tv_demographics.groupby('AGE_GROUP').size()


AGE_GROUP
18-30    14320
31-40    25972
41-50    34325
51+      60839
dtype: int64

### Non tv user by age groups

In [82]:
no_tv_demographics.groupby('AGE_GROUP').size()

C:\Users\dell\AppData\Local\Temp\ipykernel_1800\133730484.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  no_tv_demographics.groupby('AGE_GROUP').size()


AGE_GROUP
18-30     7215
31-40    10496
41-50    19045
51+      49005
dtype: int64

In [20]:
# Merging users history with channels 
tv_merged_channel_df = tv_users_df.merge(channels_df, left_on='CHANNEL_ID', right_on='CHANNEL_ID')
no_tv_merged_channel_df = no_tv_users_df.merge(channels_df, left_on='CHANNEL_ID', right_on='CHANNEL_ID')

### Number of tv users grouped by product channel 

In [8]:
tv_merged_channel_df.groupby([ 'NAME'])['USER_ID'].nunique().sort_values(ascending=False)

NAME
France 24 Anglais    2884
RTI 1                2191
ortm                 2127
ina                  2007
mango-homicide       1423
                     ... 
Histoire TV             1
mango-sonar             1
RMC Découverte          1
TvBreizh                1
wax_drama_ci            1
Name: USER_ID, Length: 208, dtype: int64

### Number of non tv users grouped by channel

In [21]:
no_tv_merged_channel_df.groupby([ 'NAME'])['USER_ID'].nunique().sort_values(ascending=False)

NAME
France 24 Anglais    2856
ina                  1831
RTI 1                1736
francetv             1121
gametoon             1000
                     ... 
luxetv4k                1
mango-sonar             1
mango-cbs               1
mango-mylove            1
wax_drama_ci            1
Name: USER_ID, Length: 201, dtype: int64